In [ ]:
!pip uninstall -q tensorflow tensorflow-estimator tensorboard tensorflow-probability 
!pip install -q tensorflow==2.1.0 

Proceed (y/n)? ㅛ
Your response ('ㅛ') was not one of the expected responses: y, n
Proceed (y/n)? ㅛ
Your response ('ㅛ') was not one of the expected responses: y, n
Proceed (y/n)? y
Proceed (y/n)? y
Proceed (y/n)? y
Proceed (y/n)? y
     |████████████████████████████████| 421.8 MB 25 kB/s 
     |████████████████████████████████| 3.8 MB 39.7 MB/s 
     |████████████████████████████████| 448 kB 61.5 MB/s 
     |████████████████████████████████| 50 kB 7.9 MB/s 


In [5]:
import tensorflow as tf 
import tensorflow.keras as keras 
import numpy as np

In [60]:
print(tf.__version__)
print(tf.test.gpu_device_name())

2.7.0
/device:GPU:0


In [1]:
with open('눈의 여왕.txt', 'r',encoding = 'utf-8') as f: EyesQueen_text = f.read()


In [2]:
len(EyesQueen_text)
#가져온 텍스트의 길이

32932

In [3]:
EyesQueen_text[-20:]

'하고 눈부시게 아름다운 여름이었다.\n'

In [6]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower = False, char_level = True) 
#lower = False 는 필수 lower는 영어에서 대문자를 소문자로 바꿔줌
#lower 에서 한국어는 초성중성종성으로 나누어줌
#텍스트들을 정수로 바꿔준다. (임의의 정수 ex - 우리 = 134)


In [7]:

tokenizer.fit_on_texts(EyesQueen_text)
#모든 merged_text들의 값들을 tokenizer

In [8]:
len(tokenizer.word_index)
#글자단위 (공백단위로 token화 해주겠다) 839개의 토큰

839

In [9]:
tokenizer.texts_to_sequences(['하고 눈부시게 아름다운 여름이었다.'])
#토큰화 된 결과들


[[21, 10, 1, 50, 102, 66, 9, 1, 14, 108, 2, 91, 1, 40, 108, 4, 17, 2, 3]]

In [10]:
tokenizer.sequences_to_texts([[21, 10, 1, 50, 102, 66, 9, 1, 14, 108, 2, 91, 1, 40, 108, 4, 17, 2, 3]])

['하 고   눈 부 시 게   아 름 다 운   여 름 이 었 다 .']

In [11]:
max_id = len(tokenizer.word_index) 
dataset_size = tokenizer.document_count

In [12]:
print(max_id)
print(dataset_size)

839
32932


In [13]:
[encoded] = np.array(tokenizer.texts_to_sequences([EyesQueen_text])) - 1

In [14]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [15]:
n_steps = 100
window_length = n_steps + 1 # target <= input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)
#TBPTT

In [16]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [17]:
np.random.seed(42)
tf.random.set_seed(42)

In [18]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [19]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
#입력데이터에 대한 원핫 인코딩

In [20]:
dataset = dataset.prefetch(1)

In [21]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)
#결과에서 100은 시간
#입력데이터 100개에 대해서 그다음의 한글자 100개의 글자 다음의 한글자 까지 맞춰야한다

(32, 100, 839) (32, 100)


In [22]:

from keras.layers import LSTM 
from keras.models import Sequential 
from keras.layers import Dense 
model = keras.Sequential()
model.add(LSTM(units=128,activation='tanh',input_shape=[None,max_id]))
model.add(Dense(max_id,activation="softmax"))


In [23]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               495616    
                                                                 
 dense (Dense)               (None, 839)               108231    
                                                                 
Total params: 603,847
Trainable params: 603,847
Non-trainable params: 0
_________________________________________________________________


In [25]:
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,epochs=10)

Epoch 1/10


InvalidArgumentError: ignored

저의 무지함에 무릎을 꿇습니다..
이번생은 틀린것일지 모르겠습니다.
더 강해지기위해 노력하겠습니다. 아직많이 약한것같습니다.
한학기동안 고생 많으셨습니다. 감사합니다..



